## Homework-10 - Network analysis of contacts in infectious disease

Ziying Wang

### Imports and Data

In [67]:
import pandas as pd
import numpy as np
from tabulate import tabulate

In [68]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

/var/folders/tn/1rpmlfx112j95v9ws4cxqb8c0000gn/T/ipykernel_15810/497376691.py:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


In [69]:
df = pd.read_csv("ia-infect-dublin.csv",header=None, )
df.columns=['Nodes','Edges']
df

,Nodes,Edges
0,6,3
1,7,2
2,9,5
3,11,8
4,12,7
...,...,...
2760,410,304
2761,410,343
2762,410,351
2763,410,362


### Question 1: Count of all direct contacts of person with id = 1 (Hint: check for person id in both columns)

In [70]:
# by nodes
df_q1_nodes = df.groupby(["Nodes"])
df_q1_nodes = df_q1_nodes.agg({'Edges' : 'count'})
df_q1_nodes

,Edges
Nodes,
6,1
7,1
9,1
11,1
12,1
...,...
406,14
407,2
408,23


In [49]:
# by edges
df_q1_edges = df.groupby(["Edges"])
df_q1_edges = df_q1_edges.agg({'Nodes' : 'count'})
df_q1_edges

,Nodes
Edges,
1,24
2,26
3,23
4,7
5,19
...,...
381,1
383,1
384,1


By looking at both Nodes and Edges column, we find that only Edges column contains direct contact with person ID = 1, which the count is **24** 

### Question 2: Degree Frequency Table: Degree = number of edges from each node - Print a table - rows for each node with number of edges originating from that node.

In [71]:
# displaying the DataFrame
print(tabulate(df_q1_nodes, headers = 'keys', tablefmt = 'psql'))

+---------+---------+
|   Nodes |   Edges |
|---------+---------|
|       6 |       1 |
|       7 |       1 |
|       9 |       1 |
|      11 |       1 |
|      12 |       1 |
|      13 |       1 |
|      14 |       3 |
|      21 |       1 |
|      22 |       1 |
|      28 |       1 |
|      29 |       2 |
|      30 |       3 |
|      31 |       1 |
|      32 |       1 |
|      35 |       1 |
|      37 |       5 |
|      38 |       1 |
|      39 |       1 |
|      40 |       2 |
|      41 |       2 |
|      43 |       3 |
|      45 |       1 |
|      49 |       3 |
|      50 |       1 |
|      51 |       3 |
|      54 |       5 |
|      55 |       2 |
|      56 |       1 |
|      57 |       3 |
|      58 |       3 |
|      59 |       1 |
|      60 |       4 |
|      61 |       3 |
|      62 |       1 |
|      63 |       4 |
|      64 |       6 |
|      65 |       2 |
|      67 |       2 |
|      68 |       2 |
|      70 |       1 |
|      71 |       1 |
|      72 |       1 |
|      73 

### Question 3: Path Calculation: For person_id = 1, list all person_ids that are connected only after 1 intermediate person.

The idea to solve this questions has three main steps:
1. find all the direct connections for person_id=1
2. find all the direct connection from results in step 1 (which these are indirect connections of person_id=1)
3. filter out all nodes that exists in both step 1 and 2

In [87]:
# utility function to find all direct connects with given dataframe and given id
def find_direct(input_df, id):
    # find direct connections based on person_id=1
    direct_connection = input_df[(input_df==id).any(axis=1)]
    # merge two rows
    direct_connection = pd.concat([direct_connection['Nodes'], direct_connection['Edges']])
    # sort the connections
    direct_connection = direct_connection.sort_values()
    # remove duplicates
    direct_connection = direct_connection.unique()
    
    return direct_connection

In [90]:
# make a dataframe for question 3
df_q3_output = pd.DataFrame(columns=['Direct Connection', 'Indirect Connections'])
df_q3_output

,Direct Connection,Indirect Connections


In [92]:
# find all direct connections for person_id = 1
direct = find_direct(df, 1)
direct

array([  1,  14,  30,  63,  78,  83,  97, 120, 125, 130, 144, 157, 174,
       185, 196, 210, 240, 274, 293, 305, 318, 328, 333, 342, 361])

In [93]:
#loop thru to find all indirect connections
for id in direct:
    # filter out all id=1 cases
    if id == 1:
        continue
    # find all direct-direct connections
    indirect = find_direct(df, id)
    # filter out all the duplicates
    indirect = indirect[~np.isin(indirect, direct)]
    # append to dataframe
    df_q3_output.loc[len(df_q3_output)] = [id, indirect]

In [94]:
df_q3_output

,Direct Connection,Indirect Connections
0,14,"[3, 5, 37, 41, 54, 90, 129, 139, 172, 225, 241, 258, 262, 301, 307, 319, 398, 401]"
1,30,"[5, 27, 38, 105, 132, 135, 150, 233, 243, 285, 297, 322, 326, 337, 364, 400]"
2,63,"[8, 34, 36, 67, 94, 121, 148, 154, 162, 178, 181, 192, 228, 255, 271, 347, 354, 357]"
3,78,"[5, 21, 44, 58, 85, 105, 173, 233, 349, 358, 364, 369]"
4,83,"[38, 60, 112, 150, 220, 285, 308, 337, 358]"
5,97,"[5, 21, 58, 85, 105, 132, 173, 364]"
6,120,"[47, 53, 60, 91, 111, 168, 177, 180, 213, 237, 259, 261, 266, 288, 300, 314, 323]"
7,125,"[23, 36, 53, 60, 91, 111, 114, 140, 189, 205, 231, 300, 314, 337, 348, 388, 408]"
8,130,"[60, 91, 140, 205, 231, 300, 348, 370, 408]"
9,144,"[5, 21, 58, 105, 150, 173, 206, 261, 326, 337, 364, 402]"
